# 🚀 Jordan RL Trading Bot - Google Colab GPU Training

**Free GPU Power for 1-Year Trading Simulations!**

This notebook clones your GitHub repo, mounts Google Drive, installs dependencies, and runs hyperparameter optimization + training with GPU acceleration.

## 📋 Prerequisites
- Upload your `jordan_data_backup.zip` to Google Drive
- Your GitHub repo: https://github.com/mmayes313/jordan-rl-trading-bot

## 1️⃣ Mount Google Drive & Clone Repository

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Clone your repository
!git clone https://github.com/mmayes313/jordan-rl-trading-bot.git
%cd jordan-rl-trading-bot

# Extract data from Drive (adjust path if needed)
!unzip -o /content/drive/MyDrive/jordan_data_backup.zip -d .

print("✅ Repository cloned and data extracted!")

## 2️⃣ Install Dependencies & Setup

In [ ]:
# Install Python dependencies
!pip install -r requirements.txt

# Install TA-Lib (required for technical indicators)
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
%cd ..
!pip install TA-Lib

# Verify GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")

print("✅ Dependencies installed and GPU ready!")

## 3️⃣ Hyperparameter Optimization (Bayesian Search)

In [ ]:
# Run hyperparameter optimization
import sys
import json
import os
sys.path.append('.')

from src.models.hyperparam_optimizer import run_optimization
from src.data.data_loader import load_historical_data
from src.environment.historical_env import HistoricalTradingEnv

# Load your exported data
try:
    data = load_historical_data('data/raw/eurusd_1y.csv')
    print(f"✅ Loaded {len(data)} data points")
except Exception as e:
    print(f"❌ Data loading failed: {e}")
    # Fallback to sample data generation
    import pandas as pd
    import numpy as np
    dates = pd.date_range('2023-01-01', periods=1000, freq='H')
    data = pd.DataFrame({
        'timestamp': dates,
        'open': 1.07 + np.random.randn(1000) * 0.01,
        'high': 1.072 + np.random.randn(1000) * 0.01,
        'low': 1.068 + np.random.randn(1000) * 0.01,
        'close': 1.071 + np.random.randn(1000) * 0.01,
        'volume': np.random.randint(1000, 2000, 1000)
    })
    print(f"✅ Generated {len(data)} sample data points")

# Create environment
env = HistoricalTradingEnv(data, asset='EURUSD')

# Run optimization (adjust n_trials as needed)
n_trials = 50  # Increase for better results (100-200 recommended)
print(f"🚀 Starting hyperparameter optimization with {n_trials} trials...")

best_params, study = run_optimization(env, n_trials=n_trials)

# Save results
os.makedirs('data/models', exist_ok=True)
with open('data/models/best_hyperparams.json', 'w') as f:
    json.dump(best_params, f, indent=4)

# Save to Google Drive
!cp data/models/best_hyperparams.json /content/drive/MyDrive/

print(f"✅ Optimization complete! Best params: {best_params}")
print("📁 Results saved to Google Drive")

## 4️⃣ Train PPO Model with Optimized Parameters

In [ ]:
# Load optimized hyperparameters
with open('data/models/best_hyperparams.json', 'r') as f:
    best_params = json.load(f)

print(f"🔧 Using optimized parameters: {best_params}")

# Import and setup model
from src.models.ppo_model import train_model
from src.environment.historical_env import HistoricalTradingEnv

# Create environment with optimized params
env = HistoricalTradingEnv(data, asset='EURUSD')

# Training configuration
total_timesteps = 500000  # Adjust based on your needs
model_save_path = 'data/models/optimized_ppo_model'

print(f"🚀 Training PPO model for {total_timesteps:,} timesteps...")

# Train the model
model = train_model(env, total_timesteps=total_timesteps, **best_params)

# Save the trained model
model.save(model_save_path)
print(f"✅ Model saved to {model_save_path}")

# Copy to Google Drive
!cp -r data/models/ /content/drive/MyDrive/jordan_models/
print("📁 Model copied to Google Drive")

## 5️⃣ Evaluate Model Performance

In [ ]:
# Load and evaluate the trained model
from stable_baselines3 import PPO
import matplotlib.pyplot as plt

# Load model
model = PPO.load(model_save_path)

# Create evaluation environment
eval_env = HistoricalTradingEnv(data, asset='EURUSD')

# Run evaluation
obs, info = eval_env.reset()
total_reward = 0
trades = []

for _ in range(len(data) - 1):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)
    total_reward += reward
    
    if action != 0:  # If a trade was made
        trades.append({
            'step': _,
            'action': action,
            'reward': reward,
            'equity': info.get('equity', 0)
        })

print(f"📊 Total Reward: {total_reward:.2f}")
print(f"📊 Number of Trades: {len(trades)}")
print(f"📊 Win Rate: {sum(1 for t in trades if t['reward'] > 0) / len(trades) * 100:.1f}%" if trades else "No trades made")

# Plot equity curve
if trades:
    equity_curve = [t['equity'] for t in trades]
    plt.figure(figsize=(12, 6))
    plt.plot(equity_curve)
    plt.title('Model Equity Curve')
    plt.xlabel('Trade Number')
    plt.ylabel('Equity')
    plt.grid(True)
    plt.show()

print("✅ Model evaluation complete!")

## 6️⃣ Download Results to Local Machine

In [ ]:
# Create results archive
!zip -r jordan_training_results.zip data/models/ notebooks/

# Download to local machine
from google.colab import files
files.download('jordan_training_results.zip')

print("📁 Download the zip file to get your trained models and results!")
print("📁 Also available in Google Drive: /MyDrive/jordan_models/")